<a href="https://colab.research.google.com/github/chiyanglin-AStar/python_stock_study/blob/main/%E8%B6%85%E7%B0%A1%E5%96%AE%E5%8F%B0%E8%82%A1%E6%AF%8F%E6%97%A5%E7%88%AC%E8%9F%B2%E6%95%99%E5%AD%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

原文網址：https://www.finlab.tw/超簡單台股每日爬蟲教學

In [ ]:
import requests
from io import StringIO
import pandas as pd
import numpy as np

datestr = '20230503'

# 下載股價
r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')

# 整理資料，變成表格
df = pd.read_csv(StringIO(r.text.replace("=", "")), 
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)

# 整理一些字串：

df.apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"), errors='coerce'))
df

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
0,0050,元大台灣50,"4,652,271","13,074","544,828,139",117.35,117.35,116.80,117.20,-,0.50,117.20,9,117.25,84,0.00,NaN
1,0051,元大中型100,"31,063",162,"1,794,331",57.90,57.90,57.65,57.75,-,0.15,57.70,1,57.75,3,0.00,NaN
2,0052,富邦科技,"194,764",334,"19,521,381",100.30,100.40,99.95,100.25,-,0.45,100.10,1,100.25,1,0.00,NaN
3,0053,元大電子,"5,136",964,"300,072",58.60,58.60,58.55,58.55,-,0.20,58.55,2,58.60,25,0.00,NaN
4,0055,元大MSCI金融,"93,699",239,"2,096,569",22.40,22.40,22.33,22.38,-,0.01,22.38,4,22.39,2,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24065,9944,新麗,"61,895",92,"1,258,116",20.35,20.45,20.25,20.45,,0.00,20.30,1,20.45,14,9.65,NaN
24066,9945,潤泰新,"2,900,702","1,773","102,786,305",35.50,35.60,35.30,35.35,-,0.15,35.35,192,35.40,67,12.19,NaN
24067,9946,三發地產,"153,608",104,"2,212,668",14.50,14.50,14.30,14.50,,0.00,14.40,7,14.50,22,33.72,NaN
24068,9955,佳龍,"4,708,578","2,480","127,711,360",27.50,28.20,26.50,26.55,+,0.30,26.50,154,26.60,1,0.00,NaN


In [ ]:
df.head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
0,0050,元大台灣50,"4,652,271","13,074","544,828,139",117.35,117.35,116.80,117.20,-,0.50,117.20,9,117.25,84,0.00,NaN
1,0051,元大中型100,"31,063",162,"1,794,331",57.90,57.90,57.65,57.75,-,0.15,57.70,1,57.75,3,0.00,NaN
2,0052,富邦科技,"194,764",334,"19,521,381",100.30,100.40,99.95,100.25,-,0.45,100.10,1,100.25,1,0.00,NaN
3,0053,元大電子,"5,136",964,"300,072",58.60,58.60,58.55,58.55,-,0.20,58.55,2,58.60,25,0.00,NaN
4,0055,元大MSCI金融,"93,699",239,"2,096,569",22.40,22.40,22.33,22.38,-,0.01,22.38,4,22.39,2,0.00,NaN


In [ ]:
import requests
from io import StringIO
import pandas as pd
import numpy as np

def crawl_price(date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '}) 
                                        for i in r.text.split('\n') 
                                        if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    ret = ret.set_index('0050')
    ret['成交金額'] = ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    return ret

抓九天資料

In [ ]:
import datetime
import time

data = {}
n_days = 2
date = datetime.datetime.now()
fail_count = 0
allow_continuous_fail_count = 5
while len(data) < n_days:

    print('parsing', date)
    # 使用 crawPrice 爬資料
    try:
        # 抓資料
        data[date.date()] = crawl_price(date)
        print('success!')
        fail_count = 0
    except:
        # 假日爬不到
        print('fail! check the date is holiday')
        fail_count += 1
        if fail_count == allow_continuous_fail_count:
            raise
            break
    
    # 減一天
    date -= datetime.timedelta(days=1)
    time.sleep(10)

parsing 2023-05-03 13:57:06.431267
fail! check the date is holiday
parsing 2023-05-02 13:57:06.431267
fail! check the date is holiday
parsing 2023-05-01 13:57:06.431267
fail! check the date is holiday
parsing 2023-04-30 13:57:06.431267
fail! check the date is holiday


KeyboardInterrupt: ignored